In [ ]:
!pip install pymorphy2
!pip install wiki-ru-wordnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import pymorphy2 as pm
import nltk
nltk.download('punkt')
import re
from wiki_ru_wordnet import WikiWordnet
from yargy import Parser, rule
from yargy.predicates import gram
from yargy.pipelines import morph_pipeline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
input_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/job/test_bewise/test_data_bewise.csv")
input_df

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага
...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...
477,5,140,client,Спасибо спасибо
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте


In [ ]:
regex = re.compile('[^а-яА-Яa-zA-Z]+')
morph = pm.MorphAnalyzer()

def normalize_texts(texts):
  normal_texts = []
  for text in texts:
    text = regex.sub(' ', text).lower().strip()
    tokenized_text = nltk.tokenize.word_tokenize(text)
    for i in range(len(tokenized_text)):
      tokenized_text[i] = morph.parse(tokenized_text[i])[0].normal_form
    normal_texts.append((' ').join(tokenized_text))
  return normal_texts

In [ ]:
manager_replicas = input_df[input_df.role == 'manager']
manager_replicas['normal_text'] = normalize_texts(manager_replicas['text'])
manager_replicas['said_hello'] = False
manager_replicas['said_bye'] = False
manager_replicas['manager_name'] = ''
manager_replicas['company_name'] = ''
manager_replicas

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,dlg_id,line_n,role,text,normal_text,said_hello,said_bye,manager_name,company_name
1,0,1,manager,Алло здравствуйте,алло здравствуйте,False,False,,
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,я звать ангелина компания диджитал бизнес звон...,False,False,,
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,угу ну возможно вы рассмотреть и другой вариан...,False,False,,
8,0,8,manager,Угу а на что вы обращаете внимание при выборе,угу а на что вы обращать внимание при выбор,False,False,,
11,0,11,manager,Что для вас приоритет,что для вы приоритет,False,False,,
...,...,...,...,...,...,...,...,...,...
472,5,135,manager,Так дмитрий,так дмитрий,False,False,,
473,5,136,manager,Все записала тогда завтра ждите звонка,всё записать тогда завтра ждать звонок,False,False,,
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,по повод виджет и с они уже обсудить конкретно...,False,False,,
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,всё я вы высылать счёт и с вы на связь если бы...,False,False,,


In [ ]:
wikiwordnet = WikiWordnet()

def get_all_synonyms(synonyms):
  all_synonyms = set()
  for synonym in synonyms:
    all_synonyms.add(synonym)
    for synset in wikiwordnet.get_synsets(synonym):
      for word in synset.get_words():
        all_synonyms.add(word.lemma())
  return all_synonyms

# a. Извлекать реплики с приветствием – где менеджер поздоровался

In [ ]:
hello_synonyms = ['здравствуйте', 'здравствуй', 'приветствую', 'добрый день', 'доброе утро', 'добрый вечер', 'доброго времени суток', 'привет', 'алло', 'салют', 'здрасьте', 'здравия желаю', 'с добрым утром',  'физкульт-привет', 'салам', 'хай', 'хелло']
all_hello_synonyms = get_all_synonyms(hello_synonyms)
normal_all_hello_synonyms = set(normalize_texts(all_hello_synonyms))

for manager_replica in manager_replicas.itertuples():
  for hello_synonym in normal_all_hello_synonyms:
    if hello_synonym in manager_replica.normal_text:
      manager_replicas.loc[manager_replica.Index, 'said_hello'] = True  

replicas_manager_said_hello = pd.DataFrame(data=manager_replicas[manager_replicas['said_hello'] == True])
replicas_manager_said_hello

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,dlg_id,line_n,role,text,normal_text,said_hello,said_bye,manager_name,company_name
1,0,1,manager,Алло здравствуйте,алло здравствуйте,True,False,,
110,1,1,manager,Алло здравствуйте,алло здравствуйте,True,False,,
166,2,2,manager,Алло здравствуйте,алло здравствуйте,True,False,,
250,3,1,manager,Алло дмитрий добрый день,алло дмитрий добрый день,True,False,,


# b. Извлекать реплики, где менеджер представил себя + c. Извлекать имя менеджера

In [ ]:
replicas_manager_represented_himself_list = []
manager_names_list = []

manager_representation_ = morph_pipeline(['меня зовут', 'зовут меня', 'мое имя', 'я'])
manager_name = rule(gram('Name'), gram('Surn').optional())

manager_representation = rule(manager_representation_, manager_name)
parser = Parser(manager_representation)

for manager_replica in manager_replicas.itertuples():
  for match in parser.findall(manager_replica.text):
    manager_replicas.loc[manager_replica.Index, 'manager_name'] = match.tokens[-1].value

replicas_manager_represented_himself = pd.DataFrame(data=manager_replicas[manager_replicas['manager_name'] != ''])
replicas_manager_represented_himself

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,dlg_id,line_n,role,text,normal_text,said_hello,said_bye,manager_name,company_name
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,я звать ангелина компания диджитал бизнес звон...,False,False,ангелина,
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес з...,я звать ангелина компания диджитал бизнес звон...,False,False,ангелина,
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес з...,я звать ангелина компания диджитал бизнес звон...,False,False,ангелина,
251,3,2,manager,Добрый меня максим зовут компания китобизнес у...,добрый я максим звать компания китобизнес удоб...,False,False,максим,


# d. Извлекать название компании

In [ ]:
company_representation = rule(morph_pipeline(['компания']), gram('NOUN'))
parser = Parser(company_representation)

for manager_replica in manager_replicas.itertuples():
  for match in parser.findall(manager_replica.text):
    manager_replicas.loc[manager_replica.Index, 'company_name'] = match.tokens[-1].value

replicas_manager_represented_company = pd.DataFrame(data=manager_replicas[manager_replicas['company_name'] != ''])
replicas_manager_represented_company

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,dlg_id,line_n,role,text,normal_text,said_hello,said_bye,manager_name,company_name
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,я звать ангелина компания диджитал бизнес звон...,False,False,ангелина,диджитал
111,1,2,manager,Меня зовут ангелина компания диджитал бизнес з...,я звать ангелина компания диджитал бизнес звон...,False,False,ангелина,диджитал
167,2,3,manager,Меня зовут ангелина компания диджитал бизнес з...,я звать ангелина компания диджитал бизнес звон...,False,False,ангелина,диджитал
251,3,2,manager,Добрый меня максим зовут компания китобизнес у...,добрый я максим звать компания китобизнес удоб...,False,False,максим,китобизнес


# e. Извлекать реплики, где менеджер попрощался

In [ ]:
bye_synonyms = ['до свидания', 'до встречи', 'прощайте', 'прощай', 'всех благ', 'всего хорошего', 'всего доброго', 'до новых встреч', 'с наилучшими пожеланиями', 'до свиданья', 'всего наилучшего', 'до скорого', 'до скорой встречи', 'до связи', 'счастливо оставаться', 'бай', 'чао', 'гуд-бай', 'ариведерче', 'покедова', 'покеда', 'бай-бай']
all_bye_synonyms = get_all_synonyms(bye_synonyms)
normal_all_bye_synonyms = set(normalize_texts(all_bye_synonyms))

for manager_replica in manager_replicas.itertuples():
  for bye_synonym in normal_all_bye_synonyms:
    if bye_synonym in manager_replica.normal_text:
      manager_replicas.loc[manager_replica.Index, 'said_bye'] = True 

replicas_manager_said_bye = pd.DataFrame(data=manager_replicas[manager_replicas['said_bye'] == True])
replicas_manager_said_bye

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,dlg_id,line_n,role,text,normal_text,said_hello,said_bye,manager_name,company_name
108,0,108,manager,Всего хорошего до свидания,весь хороший до свидание,False,True,,
162,1,53,manager,Угу да вижу я эту почту хорошо тогда исправлю ...,угу да видеть я этот почта хорошо тогда исправ...,False,True,,
163,1,54,manager,До свидания,до свидание,False,True,,
300,3,51,manager,Угу все хорошо да понедельника тогда всего доб...,угу всё хорошо да понедельник тогда весь добрый,False,True,,
335,4,33,manager,Во вторник все ну с вами да тогда до вторника ...,в вторник всё ну с вы да тогда до вторник до с...,False,True,,
479,5,142,manager,Ну до свидания хорошего вечера,ну до свидание хороший вечер,False,True,,


# f. Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

In [ ]:
dlg_id_where_said_hello = set(manager_replicas[manager_replicas['said_hello'] == True].dlg_id)
dlg_id_where_said_bye = set(manager_replicas[manager_replicas['said_bye'] == True].dlg_id)

dlg_id_where_said_hello_and_bye = dlg_id_where_said_hello & dlg_id_where_said_bye
dlg_id_where_said_hello_and_bye

{0, 1, 3}